In [3]:
import numpy as np
import cv2
import math
import json
import logging
import os

logger = logging.getLogger(__name__)

BASE_DIR = os.getcwd()
RED = (255, 0, 0)
GREEN = (0, 255,0)
YELLOW = (0, 255, 255)

def get_translation(p1, p2):
    x1,y1 = p1
    x2,y2 = p2
    dy = (y2-y1)
    dx = (x2-x1)
    return dx, dy

def get_angle(p1, p2):
    dx, dy = get_translation(p1, p2)
    dist = math.sqrt(dx + dy)
    rad = math.atan2(dy, dx)
    return np.rad2deg(rad)
    
def rotate_and_translate_image(image: np.ndarray, angle: int, center: tuple, d_x: int, d_y: int):
    """
    Rotate and translate an image.
    Params: 
        image: numpy ndarray
        angle: rotation error in degrees
        center: center point to apply the rotation
        d_x: translation along the x-axis
        d_y: translation along the y-axis
    """
    image_center = center
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    
    translation_mat = np.float32([[1, 0, d_x], [0, 1, d_y]])

    combined_mat = np.dot(rot_mat, np.vstack([translation_mat, [0, 0, 1]]))

    # Apply combined transformation
    rows, cols = image.shape[:2]
    result = cv2.warpAffine(image, combined_mat[:2, :], (cols, rows), flags=cv2.INTER_LINEAR)

    return result
def get_img_array(filename):
    array = cv2.imread(filename)
    final_array = cv2.cvtColor(array, cv2.COLOR_BGR2RGB)
    return final_array

def get_template_coords(img:np.ndarray, template_array:np.ndarray, master_coords:list, template_area_factor:float):
    """
    Get the coordinates of a template within an image crop.

    Parameters:
        img: the image being evaluated
        template_array: The template image.
        master_coords: The coordinates of the center of the template on the image.
        template_area_factor: The scaling factor for the template area.

    Returns:
        numpy.ndarray: An array containing the coordinates (X, Y) of the real 
        center of the template in the image.
    """
    t_width, t_height, _ = template_array.shape
    master_width = int(t_width * template_area_factor)
    master_height = int(t_height * template_area_factor)
    crop = img[
        master_coords[1]-master_height:master_coords[1]+master_height, 
        master_coords[0]-master_width:master_coords[0]+master_width
    ]

    result = cv2.matchTemplate(crop, template_array,cv2.TM_CCOEFF_NORMED)
    (minVal, maxVal, minLoc, maxLoc) = cv2.minMaxLoc(result)
    (startX, startY) = maxLoc
    X = startX + int(t_height/2) + master_coords[0]-master_width
    Y = startY + int(t_width/2) + master_coords[1]-master_height
    coords = np.array([X,Y])
    return coords

def get_template_coords_for_img(img: np.ndarray, refdata:dict)-> tuple:    
    master_start = [refdata['locator']['left']['x_pos'], refdata['locator']['left']['y_pos']]
    master_end = [refdata['locator']['right']['x_pos'], refdata['locator']['right']['y_pos']]

    template_img_1 = get_img_array(os.path.join(BASE_DIR, "template_left.png"))
    template_img_2 = get_img_array(os.path.join(BASE_DIR, "template_right.png"))
    template_area_factor = refdata['locator']['factor']

    x1, y1 = get_template_coords(img, template_img_1, master_start, template_area_factor)
    x2, y2 = get_template_coords(img, template_img_2, master_end, template_area_factor)
    
    ########## draw rectangles ####################
    w, h, _ = template_img_1.shape
    w *= template_area_factor
    h *= template_area_factor
    img = cv2.rectangle(img, (round(x1-w/2),round(y1-h/2)), (round(x1+w/2),round(y1+h/2)), RED, 3)

    w, h, _ = template_img_2.shape
    w *= template_area_factor
    h *= template_area_factor
    img = cv2.rectangle(img, (round(x2-w/2),round(y2-h/2)), (round(x2+w/2),round(y2+h/2)), GREEN, 3)
    ########## draw rectangles ####################
    return img,x1,y1,x2,y2

def get_rectangles(components) -> list:
    """
    Get rectangle coordinates from a components list.
    Params:
        components: a list of components

    Return:
        A dict with component id and their coordinates. Ex:
        {1:{"x_start":123, "x_end":456, "y_start":222, "y_end":333, 2:{...}}
    """
    rectangles = {}
    for component in components:
        x_pos = component.get('x_pos', None)
        y_pos = component.get('y_pos', None)
        crop_width = component.get('crop_width', None)
        crop_height = component.get('crop_height', None)
        x_start = x_pos - int(crop_width / 2)
        x_end = x_pos + int(crop_width / 2)
        y_start = y_pos - int(crop_height / 2)
        y_end = y_pos + int(crop_height / 2)
        rectangles[component['id']] = dict(x_start=x_start, x_end=x_end, y_start=y_start, y_end=y_end)
    return rectangles

def get_indexed_cropped_images(img_array, components) -> list:
    """
    Create crops from a image.
    Params:
        img_array: a image array
        components: a list of components

    Return:
        A list of tuples of crop images with the respective id.
        [(id, crop_image, component_class), (id, crop_image, component_class), ...]
    """
    rectangle_coords = get_rectangles(components)
    cropped_images = []
    for component in components:
        id = component.get('id', None)
        component_class = component.get('class')
        rectangle_coord = rectangle_coords[id]
        
        crop = img_array[rectangle_coord['y_start']: rectangle_coord['y_end'], rectangle_coord['x_start'] :rectangle_coord['x_end']]
        cropped_images.append((id, crop, component_class))
    return cropped_images

def get_ref_data(filename):
    refdata = None
    with open(filename) as data:
        refdata = json.load(data)
    return refdata


In [4]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import os
from matplotlib import pyplot as plt
import time


def realtime_test(ang=0, x=0, y=0):
    
    BASE_DIR = os.getcwd()
    img = get_img_array(os.path.join(BASE_DIR, 'original_images', 'master0.png'))
    refdata = get_ref_data(os.path.join(BASE_DIR, "silverado_referencedata.json"))
    ref_start_point = [refdata['locator']['left']['x_pos'], refdata['locator']['left']['y_pos']] 

    changed_img = rotate_and_translate_image(img, ang, tuple(ref_start_point), x, y)

    img,x1,y1,x2,y2 = get_template_coords_for_img(changed_img, refdata)
    
    fix_ang = get_angle((x1,y1),(x2,y2))
    logger.debug(f'Correction angle={fix_ang}')
    logger.debug(f'Ref coords={ref_start_point}, Templ coords={(x1,y1)}')
    fix_x, fix_y = get_translation(ref_start_point,(x1,y1))
    logger.debug(f'Correction coords=({fix_x}, {fix_y})')
    start = time.time()
    # fix the image
    final_img = rotate_and_translate_image(changed_img, fix_ang, tuple(ref_start_point), -fix_x, -fix_y)
    print(f'Elapsed time={time.time()-start}')
    f, axarr = plt.subplots(1,2)
    axarr[0].imshow(changed_img)
    axarr[1].imshow(final_img)

    refdata = get_ref_data(os.path.join(BASE_DIR, "silverado_referencedata.json"))
    components = refdata['components']
    imgs = get_indexed_cropped_images(final_img, components) 
    
    f, axarr = plt.subplots(1,4)
    for n, im in enumerate(imgs):
        axarr[n].imshow(im[1])

    
interact(realtime_test, 
 ang=widgets.IntSlider(min=-30.0, max=30.0, step=0.1, value=0.0),
 x=widgets.IntSlider(min=-120, max=120, step=1, value=-62),
 y=widgets.IntSlider(min=-120, max=120, step=1, value=-24))

interactive(children=(IntSlider(value=0, description='ang', max=30, min=-30, step=0), IntSlider(value=-62, des…

<function __main__.realtime_test(ang=0, x=0, y=0)>